In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [4]:
input_dim = 10
seq_length = 10

In [5]:
class RNN:
    def __init__(self,hidden_size,input_size):
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.U=tf.Variable(tf.random_uniform([hidden_size,input_size],dtype=tf.float32))
        self.W=tf.Variable(tf.random_uniform([hidden_size,hidden_size],dtype=tf.float32))
        self.V=tf.Variable(tf.random_uniform([input_size,hidden_size],dtype=tf.float32))

    def forward_propagation(self, x):
        # The total number of time steps
        # x : 2D [time-step value]
        T = x.shape[0].value
        init_state = tf.zeros([self.hidden_size])
        
        state_list = []
        state_list.append(init_state)
        output_list = []
        
        for t in range(1,T+1):
            
            new_state = tf.tanh(tf.add(tf.reduce_sum(tf.multiply(self.U,x[t-1]),1),tf.reduce_sum(tf.multiply(self.W,state_list[t-1]),1)))
            state_list.append(new_state)
            output = tf.reduce_sum(tf.multiply(self.W,new_state),1)
            output_list.append(output)
        
        output = tf.stack(tf.unstack(output_list),axis=0)
        state = tf.stack(tf.unstack(state_list[1:]),axis=0)
        return [output, state]
    
    def get_output(self,state):
        return self.w_hy*state+self.b_y
        
# def do_rnn(cell_list,placeholder_x,length):
#     #placeholder_x should be same as shape of cell_list
#     for i in range(length):
        

In [7]:
cell=RNN(input_dim,input_dim)

In [11]:
X = tf.placeholder(shape=(seq_length),dtype=tf.int32)
Y = tf.placeholder(shape=(seq_length),dtype=tf.int32)

one_hot = tf.one_hot(X,depth=input_dim,dtype=tf.float32)

In [12]:
one_hot

<tf.Tensor 'one_hot_1:0' shape=<unknown> dtype=float32>

In [13]:
output ,state = cell.forward_propagation(one_hot)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [889]:
output

<tf.Tensor 'stack_30:0' shape=(10, 10) dtype=float32>

In [890]:
state

<tf.Tensor 'stack_31:0' shape=(10, 10) dtype=float32>

In [891]:
decoder_logits = tf.contrib.layers.fully_connected(output,input_dim,activation_fn=None)

In [892]:
decoder_prediction = tf.argmax(decoder_logits, 1)

In [900]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(Y,depth=input_dim),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [901]:
input = np.random.uniform(0,10, (10)).round().astype(np.int).tolist()
target = np.random.uniform(0,10,(10)).round().astype(np.int).tolist()

In [902]:
input = [10, 1, 2, 9, 1, 6, 7, 9, 6, 1]

In [903]:
target = [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]

In [905]:
epoch = 5000

sess.run(tf.global_variables_initializer())
for i in range(epoch):
    l, _ = sess.run([loss,train_op],feed_dict={X:input,Y:target})
    prediction = sess.run(decoder_prediction,feed_dict={X:input,Y:target})
    asd = sess.run(output,feed_dict={X:input,Y:target})
    print("output:",target)
    print("predict:",prediction)
    print("loss:",l)
#     print("output:",asd)

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [1 9 6 6 6 6 6 6 6 6]
loss: 8.58263
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 1 1 1 1 1 1 1 1]
loss: 3.14141
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 5 5 5 5 5 5 5 5 5]
loss: 2.7628
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 2 2 2 2 2 2 2 2 2]
loss: 2.0627
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 2 9 2 2 2 2 2 2 2]
loss: 2.45313
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 9 6 9 9 9 9 9 6]
loss: 1.93912
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 9 6 6 6 6 6 6 6]
loss: 1.9096
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 8 8 6 1 6 6 6 6 6]
loss: 1.77076
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 1 6 6 6 6 6]
loss: 1.61766
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 1 1 6 9 6 1]
loss: 1.43048
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 1 5 1 9 2 1]
loss: 1.25471
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 1 5 6 9 1 1]
loss: 1.1

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00397213
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00392036
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00386985
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00382058
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00377243
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00372542
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00367947
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00363456
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00359065
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00354766
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00350562
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
pred

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00156328
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00155292
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.0015427
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00153263
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00152258
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.0015127
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00150291
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00149324
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00148366
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00147417
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.0014648
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000863355
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000859459
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000855563
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000851713
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000847912
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000844145
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000840379
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000836624
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000832916
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000829244
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000825583
output: [8, 1, 5, 6, 2, 5, 6, 9,

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000560903
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000558932
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000556997
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000555098
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000553163
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000551239
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000549363
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000547499
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000545612
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000543759
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000541895
output: [8, 1, 5, 6, 2, 5, 6, 9,

...

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000296064
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000295362
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000294649
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000293947
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000293245
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000292555
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000291841
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000291139
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000290485
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000289807
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000289129
output: [8, 1, 5, 6, 2, 5, 6, 9,

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000236407
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000235919
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000235419
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000234895
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000234419
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000233919
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000233408
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000232932
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00023242
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000231968
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000231492
output: [8, 1, 5, 6, 2, 5, 6, 9, 

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000177997
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.0001777
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000177354
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000177021
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000176723
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000176438
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000176092
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000175759
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000175461
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.00017514
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000174854
output: [8, 1, 5, 6, 2, 5, 6, 9, 1,

output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000143633
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000143407
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000143168
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000142954
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000142716
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000142466
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000142239
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000142025
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000141763
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000141549
output: [8, 1, 5, 6, 2, 5, 6, 9, 1, 2]
predict: [8 1 5 6 2 5 6 9 1 2]
loss: 0.000141322
output: [8, 1, 5, 6, 2, 5, 6, 9,

KeyboardInterrupt: 

In [ ]:
asd = tf.one_hot(Y,depth=input_dim,dtype=tf.float32)
sess.run(asd,feed_dict={Y:output})

In [812]:
sess.run(one_hot,feed_dict={X:input})

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)